In [1]:
import tabula
import re
import pandas as pd

In [2]:
# Change pdf name
df = tabula.read_pdf("20201126 Waltham Forest Privated Public Register November 2020.pdf"
                     , pages='all'
                     , pandas_options = {'header': None})

In [3]:
len(df) #should total the amount of pages in the document

76

In [4]:
class SplitFields:
    def __init__(self, field):
        self.field = field
    
    @property
    def split_list(self):
        return re.split(r'([A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2})', self.field)
    
    @property
    def property_address(self):
        return ''.join(self.split_list[:len(self.split_list)-1])
    
    @property
    def licence_holder(self):
        return self.split_list[-1]
        

In [5]:
complete_register = pd.concat(df, ignore_index=True)

# remove first line which contains header data
complete_register.drop([0], inplace=True)

# reset index after dropping row
complete_register.reset_index(drop=True, inplace=True)

# get property address from field
complete_register[5] = complete_register[1].apply(lambda x: SplitFields(x).property_address)

# get licence holder from field
complete_register[6] = complete_register.apply(lambda x: SplitFields(x[1]).licence_holder if pd.isnull(x[2]) else x[2], axis=1)

In [6]:
# created a df with columns to keep
final_df = complete_register[[0,5,6,3,4]].copy()

In [7]:
# rename columns
final_df.columns = ['ref_no', 'property_address', 'licence_holder', 'start_date', 'end_date']

In [8]:
final_df.head()

,ref_no,property_address,licence_holder,start_date,end_date
0,14531,"43 Chester Road, Walthamstow, E17 7HP",Mr Lahrie Mohamed,15/03/2018,15/03/2023
1,20502,"Flat 1, 8 Goldsmith Road, Leyton, E10 5HA",Mr Lahrie Mohamed,12/05/2017,12/05/2022
2,20881,"Flat 2, 79 Park Road, Leyton, E10 7BZ",Mr Lahrie Mohamed,01/11/2018,01/11/2023
3,20932,"Ground Floor Flat, 23 Cornwallis Road, Waltham...",Mr Lahrie Mohamed,26/10/2018,26/10/2023
4,21254,"Flat 3, 62 St James Street, Walthamstow, E17 7PE",Mr Lahrie Mohamed,26/10/2018,26/10/2023


In [9]:
len(final_df)

2332